In [6]:
# !pip install pyarrow

In [60]:
import pandas as pd

# Read a Parquet file
df = pd.read_parquet('22343/239181.parquet')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16290 entries, 0 to 16289
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frame           16290 non-null  int16  
 1   row_id          16290 non-null  object 
 2   type            16290 non-null  object 
 3   landmark_index  16290 non-null  int16  
 4   x               15660 non-null  float64
 5   y               15660 non-null  float64
 6   z               15660 non-null  float64
dtypes: float64(3), int16(2), object(2)
memory usage: 700.1+ KB


In [64]:
df.head(10000)

,frame,row_id,type,landmark_index,x,y,z
0,21,21-face-0,face,0,0.451934,0.446403,-0.048092
1,21,21-face-1,face,1,0.441165,0.405696,-0.067456
2,21,21-face-2,face,2,0.445987,0.419076,-0.040718
3,21,21-face-3,face,3,0.427772,0.379115,-0.041207
4,21,21-face-4,face,4,0.439213,0.395850,-0.069420
...,...,...,...,...,...,...,...
9995,39,39-face-221,face,221,0.399564,0.350762,0.018855
9996,39,39-face-222,face,222,0.377745,0.345047,0.021205
9997,39,39-face-223,face,223,0.359383,0.345024,0.025551
9998,39,39-face-224,face,224,0.345730,0.348526,0.033537


In [65]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils

# Initialize video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Holistic
    results = holistic.process(rgb_frame)

    # Draw landmarks on the frame
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    
    # Use FACEMESH_CONTOURS for face landmarks
    mp_drawing.draw_landmarks(frame, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_CONTOURS)
    
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # Display the output
    cv2.imshow('MediaPipe Holistic', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Destroy all OpenCV windows
cv2.destroyAllWindows()


In [3]:
import cv2
import mediapipe as mp
import pandas as pd

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Initialize video capture
cap = cv2.VideoCapture(0)

# Create a DataFrame to store the landmark data
columns = ['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z']
landmark_df = pd.DataFrame(columns=columns)

while cap.isOpened():
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Holistic
    results = holistic.process(rgb_frame)

    # Extract and store face landmarks
    if results.face_landmarks:
        for i, landmark in enumerate(results.face_landmarks.landmark):
            landmark_df = landmark_df.append({
                'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
                'row_id': f"{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}_face_{i}",
                'type': 'face',
                'landmark_index': i,
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z if hasattr(landmark, 'z') else None
            }, ignore_index=True)

    # Extract and store left hand landmarks
    if results.left_hand_landmarks:
        for i, landmark in enumerate(results.left_hand_landmarks.landmark):
            landmark_df = landmark_df.append({
                'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
                'row_id': f"{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}_left_hand_{i}",
                'type': 'left_hand',
                'landmark_index': i,
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z if hasattr(landmark, 'z') else None
            }, ignore_index=True)

    # Extract and store pose landmarks
    if results.pose_landmarks:
        for i, landmark in enumerate(results.pose_landmarks.landmark):
            landmark_df = landmark_df.append({
                'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
                'row_id': f"{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}_pose_{i}",
                'type': 'pose',
                'landmark_index': i,
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z if hasattr(landmark, 'z') else None
            }, ignore_index=True)

    # Extract and store right hand landmarks
    if results.right_hand_landmarks:
        for i, landmark in enumerate(results.right_hand_landmarks.landmark):
            landmark_df = landmark_df.append({
                'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
                'row_id': f"{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}_right_hand_{i}",
                'type': 'right_hand',
                'landmark_index': i,
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z if hasattr(landmark, 'z') else None
            }, ignore_index=True)

    # Display the output
    cv2.imshow('MediaPipe Holistic', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Convert DataFrame to Parquet and save it
landmark_df.to_parquet('landmarks.parquet', index=False)

# Destroy all OpenCV windows
cv2.destroyAllWindows()


C:\Users\Animesh\AppData\Local\Temp\ipykernel_20052\2555310626.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  landmark_df = landmark_df.append({
C:\Users\Animesh\AppData\Local\Temp\ipykernel_20052\2555310626.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  landmark_df = landmark_df.append({
C:\Users\Animesh\AppData\Local\Temp\ipykernel_20052\2555310626.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  landmark_df = landmark_df.append({
C:\Users\Animesh\AppData\Local\Temp\ipykernel_20052\2555310626.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  landmark_df = landmark_df.append({
C:\Users\Animesh\AppData\Local\Temp\ipyk

In [4]:
dfn = pd.read_parquet('landmarks.parquet')

In [5]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5742 entries, 0 to 5741
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frame           5742 non-null   float64
 1   row_id          5742 non-null   object 
 2   type            5742 non-null   object 
 3   landmark_index  5742 non-null   int64  
 4   x               5742 non-null   float64
 5   y               5742 non-null   float64
 6   z               5742 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 314.1+ KB


In [9]:
dfn.head(505)

,frame,row_id,type,landmark_index,x,y,z
0,0.0,0_face_0,face,0,0.612244,0.384678,-0.041016
1,0.0,0_face_1,face,1,0.617610,0.331873,-0.065472
2,0.0,0_face_2,face,2,0.614752,0.350203,-0.037044
3,0.0,0_face_3,face,3,0.609445,0.275282,-0.045106
4,0.0,0_face_4,face,4,0.618870,0.314017,-0.068327
...,...,...,...,...,...,...,...
500,0.0,0_pose_32,pose,32,0.375277,3.568311,-0.570717
501,0.0,0_face_0,face,0,0.608957,0.386071,-0.042115
502,0.0,0_face_1,face,1,0.615179,0.332865,-0.066597
503,0.0,0_face_2,face,2,0.612454,0.352096,-0.037645


In [66]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Initialize video capture
cap = cv2.VideoCapture(0)

# Create a DataFrame to store the landmark data
columns = ['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z']
landmark_df = pd.DataFrame(columns=columns)

# Function to add landmarks to the DataFrame
def add_landmarks(type_str, landmarks, total_landmarks , frame_number):
    for i in range(total_landmarks):
        if landmarks is not None and i < len(landmarks.landmark):
            landmark = landmarks.landmark[i]
            landmark_df.loc[len(landmark_df)] = {
                'frame': frame_number,
                'row_id': f"{frame_number}_{type_str}_{i}",
                'type': type_str,
                'landmark_index': i,
                'x': landmark.x if landmark.HasField('x') else np.nan,
                'y': landmark.y if landmark.HasField('y') else np.nan,
                'z': landmark.z if landmark.HasField('z') else np.nan
            }
        else:
            # Fill in with NaN for missing landmarks
            landmark_df.loc[len(landmark_df)] = {
                'frame': frame_number,
                'row_id': f"{frame_number}_{type_str}_{i}",
                'type': type_str,
                'landmark_index': i,
                'x': np.nan,
                'y': np.nan,
                'z': np.nan
            }


frame_number=0
while cap.isOpened():
    
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Update the frame number
    frame_number +=1


    # Print frame read status and frame number
    print("Frame read status:", ret)
    print("Current frame number:", frame_number)

    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Holistic
    results = holistic.process(rgb_frame)

    # Add face landmarks (468 in total)
    add_landmarks('face', results.face_landmarks, 468, frame_number)

    # Add left hand landmarks (21 in total)
    add_landmarks('left_hand', results.left_hand_landmarks, 21,frame_number)

    # Add pose landmarks (33 in total)
    add_landmarks('pose', results.pose_landmarks, 33, frame_number)

    # Add right hand landmarks (21 in total)
    add_landmarks('right_hand', results.right_hand_landmarks, 21, frame_number)


    # Display the output
    cv2.imshow('MediaPipe Holistic', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Convert DataFrame to Parquet and save it
landmark_df.to_parquet('landmarks.parquet', index=False)

# Destroy all OpenCV windows
cv2.destroyAllWindows()


Frame read status: True
Current frame number: 1
Frame read status: True
Current frame number: 2
Frame read status: True
Current frame number: 3
Frame read status: True
Current frame number: 4
Frame read status: True
Current frame number: 5
Frame read status: True
Current frame number: 6
Frame read status: True
Current frame number: 7
Frame read status: True
Current frame number: 8
Frame read status: True
Current frame number: 9


In [70]:
dfn = pd.read_parquet('landmarks.parquet')
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4887 entries, 0 to 4886
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frame           4887 non-null   int64  
 1   row_id          4887 non-null   object 
 2   type            4887 non-null   object 
 3   landmark_index  4887 non-null   int64  
 4   x               4740 non-null   float64
 5   y               4740 non-null   float64
 6   z               4740 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 267.4+ KB


In [71]:
dfn.head(470)

,frame,row_id,type,landmark_index,x,y,z
0,1,1_face_0,face,0,0.542275,0.513665,-0.039683
1,1,1_face_1,face,1,0.534631,0.463230,-0.057196
2,1,1_face_2,face,2,0.538531,0.483082,-0.034494
3,1,1_face_3,face,3,0.521578,0.422387,-0.035445
4,1,1_face_4,face,4,0.532698,0.447956,-0.058845
...,...,...,...,...,...,...,...
465,1,1_face_465,face,465,0.549500,0.395921,0.003291
466,1,1_face_466,face,466,0.609351,0.381508,0.020987
467,1,1_face_467,face,467,0.614789,0.373296,0.021636
468,1,1_left_hand_0,left_hand,0,NaN,NaN,NaN


In [72]:
dfn.head(490)

,frame,row_id,type,landmark_index,x,y,z
0,1,1_face_0,face,0,0.542275,0.513665,-0.039683
1,1,1_face_1,face,1,0.534631,0.463230,-0.057196
2,1,1_face_2,face,2,0.538531,0.483082,-0.034494
3,1,1_face_3,face,3,0.521578,0.422387,-0.035445
4,1,1_face_4,face,4,0.532698,0.447956,-0.058845
...,...,...,...,...,...,...,...
485,1,1_left_hand_17,left_hand,17,NaN,NaN,NaN
486,1,1_left_hand_18,left_hand,18,NaN,NaN,NaN
487,1,1_left_hand_19,left_hand,19,NaN,NaN,NaN
488,1,1_left_hand_20,left_hand,20,NaN,NaN,NaN


In [73]:
dfn.head(525)

,frame,row_id,type,landmark_index,x,y,z
0,1,1_face_0,face,0,0.542275,0.513665,-0.039683
1,1,1_face_1,face,1,0.534631,0.463230,-0.057196
2,1,1_face_2,face,2,0.538531,0.483082,-0.034494
3,1,1_face_3,face,3,0.521578,0.422387,-0.035445
4,1,1_face_4,face,4,0.532698,0.447956,-0.058845
...,...,...,...,...,...,...,...
520,1,1_pose_31,pose,31,0.672068,3.345926,-0.194274
521,1,1_pose_32,pose,32,0.452018,3.327996,-0.399972
522,1,1_right_hand_0,right_hand,0,NaN,NaN,NaN
523,1,1_right_hand_1,right_hand,1,NaN,NaN,NaN


In [74]:
dfn.head(545)

,frame,row_id,type,landmark_index,x,y,z
0,1,1_face_0,face,0,0.542275,0.513665,-0.039683
1,1,1_face_1,face,1,0.534631,0.463230,-0.057196
2,1,1_face_2,face,2,0.538531,0.483082,-0.034494
3,1,1_face_3,face,3,0.521578,0.422387,-0.035445
4,1,1_face_4,face,4,0.532698,0.447956,-0.058845
...,...,...,...,...,...,...,...
540,1,1_right_hand_18,right_hand,18,NaN,NaN,NaN
541,1,1_right_hand_19,right_hand,19,NaN,NaN,NaN
542,1,1_right_hand_20,right_hand,20,NaN,NaN,NaN
543,2,2_face_0,face,0,0.546013,0.515294,-0.041510


In [55]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4887 entries, 0 to 4886
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frame           4887 non-null   int64  
 1   row_id          4887 non-null   object 
 2   type            4887 non-null   object 
 3   landmark_index  4887 non-null   int64  
 4   x               4719 non-null   float64
 5   y               4719 non-null   float64
 6   z               4719 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 267.4+ KB


In [75]:
dfn

,frame,row_id,type,landmark_index,x,y,z
0,1,1_face_0,face,0,0.542275,0.513665,-0.039683
1,1,1_face_1,face,1,0.534631,0.463230,-0.057196
2,1,1_face_2,face,2,0.538531,0.483082,-0.034494
3,1,1_face_3,face,3,0.521578,0.422387,-0.035445
4,1,1_face_4,face,4,0.532698,0.447956,-0.058845
...,...,...,...,...,...,...,...
4882,9,9_right_hand_16,right_hand,16,0.077627,0.519886,-0.012320
4883,9,9_right_hand_17,right_hand,17,0.005494,0.767379,-0.000386
4884,9,9_right_hand_18,right_hand,18,-0.000363,0.656578,-0.004289
4885,9,9_right_hand_19,right_hand,19,0.017923,0.600491,-0.003094


In [43]:
print(cv2.__version__)

4.8.1
